## Commands To Start AutoDock

In [ ]:
import subprocess
import urllib.request
import sys
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
import json
import subprocess
subprocess.run('pip install google-cloud-compute notebook'.split(" "), text=True, capture_output=True)
import urllib.request
from __future__ import annotations

import sys
from typing import Any

from google.api_core.extended_operation import ExtendedOperation
from google.cloud import compute_v1

###  Run the below cell and copy its output to a new cell and run it

In [ ]:
%%javascript
// Send data

fetch( 'https://api64.ipify.org/')
  .then(
    response => response.text()
  ).then(
    text => element.append("ip='"+text+"'")
  );

In [ ]:
ip=''

<div class="alert alert-block alert-danger"> <b>Attention:</b> Make sure to assign the returned ip address to the ip variable.</a>. </div> <br>

### Run the below to create a firewall rule to allow access to the GUI

In [ ]:
firewall_rule_capture = subprocess.run('gcloud compute firewall-rules list'.split(" "), text=True, capture_output=True) 
rules = firewall_rule_capture.stdout.split("\n")
rule_found = False
for x in rules:
    if 'allow-gui-accessjp32' in x:
        rule_found = True
        print("rule already exist")
if not rule_found:
    subprocess.run(['gcloud','compute','firewall-rules','create','allow-gui-accessjp32','--allow=tcp:8081','--allow=tcp:8082','--source-ranges='+ip+'/32','--description="allow-gui-access"'])

### Finally run this block

In [ ]:
url = "http://metadata.google.internal/computeMetadata/v1/instance/name"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
machine_name = urllib.request.urlopen(req).read().decode()
url = "http://metadata.google.internal/computeMetadata/v1/instance/zone"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
zone = urllib.request.urlopen(req).read().decode().split("/")[-1]
url = "http://metadata.google.internal/computeMetadata/v1/project/project-id"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
project_id = urllib.request.urlopen(req).read().decode()
instance_client = compute_v1.InstancesClient()
i = instance_client.get(project=project_id, zone=zone, instance=machine_name)
ip = i.network_interfaces[0].access_configs[0].nat_i_p

# Directly launch AutoDock Docker container
docker_out = subprocess.Popen(
    ('docker run --rm -v /home/jupyter:/config/bucket -p 8082:3000 us-east4-docker.pkg.dev/nih-cl-shared-resources/nigms-sandbox/autodock-gui').split(" ")
)


display(HTML(f"Please access AutoDock at <a href=\"http://{ip}:8082\">http://{ip}:8082</a><br>Note, it may take 30 seconds for it to fully start"))